In [68]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyRegressor,DummyClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVR, SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from tqdm.notebook import tqdm
from sklearn.metrics import root_mean_squared_error,accuracy_score, f1_score, classification_report, make_scorer
import joblib
import httpx
from bs4 import BeautifulSoup
import asyncio
import re
from tqdm.notebook import tqdm
import json
from time import sleep

In [69]:
df = pd.read_csv('../datasets/epi_r.csv')

In [70]:
df

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20047,Parmesan Puffs,3.125,28.0,2.0,2.0,64.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,671.0,22.0,28.0,583.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,563.0,31.0,38.0,652.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,631.0,45.0,24.0,517.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
ingredients = df.columns.tolist()
print(ingredients)

['title', 'rating', 'calories', 'protein', 'fat', 'sodium', '#cakeweek', '#wasteless', '22-minute meals', '3-ingredient recipes', '30 days of groceries', 'advance prep required', 'alabama', 'alaska', 'alcoholic', 'almond', 'amaretto', 'anchovy', 'anise', 'anniversary', 'anthony bourdain', 'aperitif', 'appetizer', 'apple', 'apple juice', 'apricot', 'arizona', 'artichoke', 'arugula', 'asian pear', 'asparagus', 'aspen', 'atlanta', 'australia', 'avocado', 'back to school', 'backyard bbq', 'bacon', 'bake', 'banana', 'barley', 'basil', 'bass', 'bastille day', 'bean', 'beef', 'beef rib', 'beef shank', 'beef tenderloin', 'beer', 'beet', 'bell pepper', 'berry', 'beverly hills', 'birthday', 'biscuit', 'bitters', 'blackberry', 'blender', 'blue cheese', 'blueberry', 'boil', 'bok choy', 'bon appétit', 'bon app��tit', 'boston', 'bourbon', 'braise', 'bran', 'brandy', 'bread', 'breadcrumbs', 'breakfast', 'brie', 'brine', 'brisket', 'broccoli', 'broccoli rabe', 'broil', 'brooklyn', 'brown rice', 'brown

In [72]:
data= df[['title','rating', 'almond', 'amaretto', 'anchovy', 'anise', 'aperitif', 'apple', 'apple juice', 'apricot', 'artichoke', 'arugula', 'asian pear', 'asparagus', 'aspen', 'avocado', 'bacon', 'banana', 'barley', 'basil', 'bass', 'bean', 'beef', 'beef rib', 'beef shank', 'beef tenderloin', 'beer', 'beet', 'bell pepper', 'berry', 'biscuit', 'blackberry', 'blue cheese', 'blueberry', 'bok choy', 'bourbon', 'braise', 'bran', 'brandy', 'bread', 'breadcrumbs', 'brie', 'brine', 'brisket', 'broccoli', 'broccoli rabe', 'broil', 'brown rice', 'brownie', 'brussel sprout', 'bulgur', 'burrito', 'butter', 'buttermilk', 'butternut squash', 'butterscotch/caramel', 'cabbage', 'campari', 'candy', 'cantaloupe', 'capers', 'caraway', 'cardamom', 'carrot', 'cashew', 'cauliflower', 'caviar', 'celery', 'chambord', 'champagne', 'chard', 'chartreuse', 'cheddar', 'cheese', 'cherry', 'chestnut', 'chicken', 'chickpea', 'chile', 'chile pepper', 'chili', 'chive', 'chocolate', 'cilantro', 'cinnamon', 'clam', 'clove', 'cobbler/crumble', 'coconut', 'cod', 'coffee', 'cognac/armagnac', 'collard greens', 'condiment', 'condiment/spread', 'cookie', 'cookies', 'coriander', 'corn', 'cornmeal', 'cottage cheese', 'couscous', 'crab', 'cranberry', 'cranberry sauce', 'cream cheese', 'créme de cacao', 'crêpe', 'cucumber', 'cumin', 'currant', 'curry', 'custard', 'date', 'digestif', 'dill', 'dinner', 'breakfast', 'lunch', 'dip', 'dried fruit', 'duck', 'egg', 'egg nog', 'eggplant', 'endive', 'escarole', 'fennel', 'feta', 'fig', 'fish', 'flat bread', 'fontina', 'fortified wine', 'frangelico', 'frittata', 'fruit', 'fruit juice', 'garlic', 'gin', 'ginger', 'goat cheese', 'goose', 'gouda', 'grains', 'granola', 'grape', 'grapefruit', 'grappa', 'green bean', 'green onion/scallion', 'ground beef', 'guava', 'halibut', 'ham', 'hazelnut', 'herb', 'hominy/cornmeal/masa', 'honey', 'honeydew', 'horseradish', 'hot pepper', 'hummus', 'ice cream', 'jalapeño', 'jam or jelly', 'jerusalem artichoke', 'jícama', 'kahlúa', 'kale', 'kirsch', 'kiwi', 'kumquat', 'kwanzaa', 'lamb', 'lamb chop', 'lamb shank', 'lasagna', 'leafy green', 'leek', 'legume', 'lemon', 'lemon juice', 'lemongrass', 'lentil', 'lettuce', 'lima bean', 'lime', 'lime juice', 'lingonberry', 'liqueur', 'lobster', 'lychee', 'macadamia nut', 'mandoline', 'mango', 'maple syrup', 'margarita', 'marinade', 'marsala', 'marscarpone', 'marshmallow', 'martini', 'mayonnaise', 'meat', 'meatball', 'meatloaf', 'melon', 'mezcal', 'milk/cream', 'mint', 'molasses', 'monterey jack', 'mozzarella', 'muffin', 'mushroom', 'mussel', 'mustard', 'mustard greens', 'nectarine', 'noodle', 'nut', 'nutmeg', 'oat', 'oatmeal', 'octopus', 'olive', 'onion', 'orange', 'orange juice', 'oregano', 'orzo', 'oyster', 'papaya', 'paprika', 'parmesan', 'parsley', 'passion fruit', 'pea', 'peach', 'peanut', 'peanut butter', 'peanut free', 'pear', 'pecan', 'pepper', 'pernod', 'persimmon', 'phyllo/puff pastry dough', 'pickles', 'pine nut', 'pineapple', 'pistachio', 'plantain', 'plum', 'pomegranate', 'pomegranate juice', 'poppy', 'pork', 'pork chop', 'pork rib', 'pork tenderloin', 'potato', 'poultry', 'poultry sausage', 'prosciutto', 'prune', 'pumpkin', 'punch', 'quail', 'quince', 'quinoa', 'rabbit', 'rack of lamb', 'radicchio', 'radish', 'raisin', 'raspberry', 'red wine', 'rhubarb', 'rice', 'ricotta', 'roast', 'root vegetable', 'rosemary', 'rosh hashanah/yom kippur', 'rosé', 'rub', 'rum', 'rutabaga', 'rye', 'saffron', 'sage', 'sake', 'salad', 'salad dressing', 'salmon', 'salsa', 'sangria', 'sardine', 'sauce', 'sausage', 'sauté', 'scallop', 'scotch', 'seafood', 'seattle', 'seed', 'self', 'semolina', 'sesame', 'sesame oil', 'shallot', 'shavuot', 'shellfish', 'sherry', 'shrimp', 'snapper', 'sorbet', 'soufflé/meringue', 'soup/stew', 'sour cream', 'sourdough', 'soy', 'soy sauce', 'sparkling wine', 'spice', 'spinach', 'spritzer', 'squid', 'steak', 'stew', 'stock', 'strawberry', 'stuffing/dressing', 'sugar snap pea', 'sweet potato/yam', 'swiss cheese', 'swordfish', 'taco', 'tamarind', 'tangerine', 'tapioca', 'tarragon', 'tart', 'tea', 'tequila', 'thyme', 'tilapia', 'tofu', 'tomatillo','tomato', 'tortillas', 'tree nut', 'tropical fruit', 'trout', 'tuna', 'turnip', 'vanilla', 'veal', 'vegetable', 'venison', 'vermont', 'vermouth', 'vinegar', 'vodka', 'waffle', 'walnut', 'wasabi', 'watermelon', 'wheat/gluten-free', 'whiskey', 'white wine', 'whole wheat', 'wild rice', 'wine', 'yellow squash', 'yogurt', 'zucchini', 'turkey']]


In [73]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20052 entries, 0 to 20051
Columns: 380 entries, title to turkey
dtypes: float64(379), object(1)
memory usage: 58.1+ MB


In [74]:
data = data.drop_duplicates(subset='title')
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17736 entries, 0 to 20051
Columns: 380 entries, title to turkey
dtypes: float64(379), object(1)
memory usage: 51.6+ MB


In [75]:
for col in data.select_dtypes(include='float64').columns:
    if data[col].nunique() == 2:
        data[col] = data[col].fillna(0.0)
    else:
        data[col] = data[col].fillna(data[col].median())

In [76]:
null_data = pd.DataFrame(data.isnull().sum(), columns=['amount'])
null_data[null_data['amount'] != 0]

,amount


In [77]:
data

,title,rating,almond,amaretto,anchovy,anise,aperitif,apple,apple juice,apricot,...,wheat/gluten-free,whiskey,white wine,whole wheat,wild rice,wine,yellow squash,yogurt,zucchini,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20046,Chinese Barbecued Spareribs,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20048,Artichoke and Parmesan Risotto,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20049,Turkey Cream Puff Pie,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20050,Snapper on Angel Hair with Citrus Cream,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
data.to_csv('recipes.csv')

In [60]:
X = data.drop(columns=['title','rating','breakfast','lunch','dinner'])
y = data['rating']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=21,shuffle=True)

## Regression

In [13]:
dummy_reg = DummyRegressor(strategy='mean')
dummy_reg.fit(X_train, y_train)
y_dummy = dummy_reg.predict(X_test)
rmse_dummy = root_mean_squared_error(y_test, y_dummy)
print(rmse_dummy)

1.2901824957877597


In [14]:
models_reg = {'LinearRegression': LinearRegression(),
              'Lasso': Lasso(random_state=21),
              'Ridge': Ridge(random_state=21,),
              'ElasticNet': ElasticNet(random_state=21),
              'DecisionTree': DecisionTreeRegressor(random_state=21,max_depth=5),
              'KNN': KNeighborsRegressor(),
              'Random Forest': RandomForestRegressor(random_state=21, n_estimators=100,max_depth=10,min_samples_leaf=5),
              'SVR': SVR(),
              'Gradient Boosting': GradientBoostingRegressor(random_state=21,n_estimators=100,max_depth=3)}

result=[]
cv = KFold(n_splits=5,shuffle=True,random_state=21)
for name, model in models_reg.items():
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='neg_root_mean_squared_error')
    rmse = np.round((-scores).mean(),5)
    result.append({'Model': name, 'RMSE': rmse, 'std': np.round(scores.std(),5)})
result_df = pd.DataFrame(result).sort_values(by='RMSE',ascending=True)
result_df

,Model,RMSE,std
2,Ridge,1.26118,0.02117
8,Gradient Boosting,1.26243,0.02283
6,Random Forest,1.26557,0.02403
0,LinearRegression,1.26695,0.02114
4,DecisionTree,1.28892,0.02797
7,SVR,1.29447,0.02842
1,Lasso,1.32790,0.02891
3,ElasticNet,1.32790,0.02891
5,KNN,1.38445,0.02682


In [15]:
ridge_grid = {'alpha': [0.01, 0.1, 1, 10, 100, 1000],
                 'fit_intercept': [True, False],
                 'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag'],
                 'max_iter': [100, 500, 1000, 2000]}
gs_ridge = GridSearchCV(estimator=Ridge(random_state=21), param_grid=ridge_grid, scoring='neg_root_mean_squared_error', cv=3, verbose=2, n_jobs=-1)
gs_ridge.fit(X_train,y_train)
best_params = gs_ridge.best_params_
print('best params',best_params)
print('best score', -gs_ridge.best_score_)

Fitting 3 folds for each of 288 candidates, totalling 864 fits
[CV] END alpha=0.01, fit_intercept=True, max_iter=100, solver=auto; total time=   0.5s
[CV] END alpha=0.01, fit_intercept=True, max_iter=100, solver=auto; total time=   0.4s
[CV] END alpha=0.01, fit_intercept=True, max_iter=100, solver=cholesky; total time=   0.4s
[CV] END alpha=0.01, fit_intercept=True, max_iter=100, solver=auto; total time=   0.5s
[CV] END alpha=0.01, fit_intercept=True, max_iter=100, solver=cholesky; total time=   0.5s
[CV] END alpha=0.01, fit_intercept=True, max_iter=100, solver=cholesky; total time=   0.4s
[CV] END alpha=0.01, fit_intercept=True, max_iter=100, solver=svd; total time=   1.8s
[CV] END alpha=0.01, fit_intercept=True, max_iter=100, solver=lsqr; total time=   1.2s
[CV] END alpha=0.01, fit_intercept=True, max_iter=100, solver=svd; total time=   2.1s
[CV] END alpha=0.01, fit_intercept=True, max_iter=100, solver=lsqr; total time=   1.3s
[CV] END alpha=0.01, fit_intercept=True, max_iter=100, so

/home/ivan/projects/DSboot/.venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ivan/projects/DSboot/.venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ivan/projects/DSboot/.venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END alpha=0.01, fit_intercept=True, max_iter=100, solver=sag; total time=   8.0s
[CV] END alpha=0.01, fit_intercept=True, max_iter=100, solver=sag; total time=   7.8s
[CV] END alpha=0.01, fit_intercept=True, max_iter=1000, solver=auto; total time=   0.3s
[CV] END alpha=0.01, fit_intercept=True, max_iter=1000, solver=auto; total time=   0.3s
[CV] END alpha=0.01, fit_intercept=True, max_iter=1000, solver=auto; total time=   0.3s
[CV] END alpha=0.01, fit_intercept=True, max_iter=1000, solver=svd; total time=   1.3s
[CV] END alpha=0.01, fit_intercept=True, max_iter=1000, solver=svd; total time=   1.7s
[CV] END alpha=0.01, fit_intercept=True, max_iter=1000, solver=cholesky; total time=   0.4s
[CV] END alpha=0.01, fit_intercept=True, max_iter=1000, solver=svd; total time=   1.4s
[CV] END alpha=0.01, fit_intercept=True, max_iter=1000, solver=cholesky; total time=   0.3s
[CV] END alpha=0.01, fit_intercept=True, max_iter=1000, solver=cholesky; total time=   0.3s
[CV] END alpha=0.01, fit_in

/home/ivan/projects/DSboot/.venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END alpha=0.01, fit_intercept=False, max_iter=100, solver=sag; total time=   7.6s
[CV] END alpha=0.01, fit_intercept=False, max_iter=500, solver=lsqr; total time=   0.7s
[CV] END alpha=0.01, fit_intercept=False, max_iter=500, solver=lsqr; total time=   0.7s


...

[CV] END alpha=0.1, fit_intercept=True, max_iter=500, solver=lsqr; total time=   0.8s
[CV] END alpha=0.1, fit_intercept=True, max_iter=500, solver=lsqr; total time=   0.9s
[CV] END alpha=0.1, fit_intercept=True, max_iter=500, solver=lsqr; total time=   0.8s
[CV] END alpha=0.1, fit_intercept=True, max_iter=500, solver=sparse_cg; total time=   1.5s
[CV] END alpha=0.1, fit_intercept=True, max_iter=500, solver=sparse_cg; total time=   1.6s
[CV] END alpha=0.1, fit_intercept=True, max_iter=500, solver=sparse_cg; total time=   1.6s
[CV] END alpha=0.1, fit_intercept=True, max_iter=1000, solver=auto; total time=   0.4s
[CV] END alpha=0.1, fit_intercept=True, max_iter=1000, solver=auto; total time=   0.4s
[CV] END alpha=0.1, fit_intercept=True, max_iter=1000, solver=auto; total time=   0.4s
[CV] END alpha=0.1, fit_intercept=True, max_iter=1000, solver=svd; total time=   1.6s
[CV] END alpha=0.1, fit_intercept=True, max_iter=1000, solver=svd; total time=   1.5s
[CV] END alpha=0.1, fit_intercept=Tr

/home/ivan/projects/DSboot/.venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/ivan/projects/DSboot/.venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END alpha=0.1, fit_intercept=False, max_iter=500, solver=lsqr; total time=   0.9s
[CV] END alpha=0.1, fit_intercept=False, max_iter=100, solver=sag; total time=   7.3s
[CV] END alpha=0.1, fit_intercept=False, max_iter=500, solver=sparse_cg; total time=   1.0s
[CV] END alpha=0.1, fit_intercept=False, max_iter=100, solver=sag; total time=   7.8s


/home/ivan/projects/DSboot/.venv/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END alpha=0.1, fit_intercept=False, max_iter=500, solver=sparse_cg; total time=   1.1s
[CV] END alpha=0.1, fit_intercept=False, max_iter=500, solver=sparse_cg; total time=   0.9s
[CV] END alpha=0.1, fit_intercept=False, max_iter=1000, solver=auto; total time=   0.3s
[CV] END alpha=0.1, fit_intercept=False, max_iter=1000, solver=auto; total time=   0.3s
[CV] END alpha=0.1, fit_intercept=False, max_iter=1000, solver=auto; total time=   0.3s
[CV] END alpha=0.1, fit_intercept=False, max_iter=1000, solver=svd; total time=   1.4s
[CV] END alpha=0.1, fit_intercept=False, max_iter=1000, solver=svd; total time=   1.5s
[CV] END alpha=0.1, fit_intercept=False, max_iter=1000, solver=cholesky; total time=   0.4s
[CV] END alpha=0.1, fit_intercept=False, max_iter=1000, solver=svd; total time=   1.6s
[CV] END alpha=0.1, fit_intercept=False, max_iter=1000, solver=cholesky; total time=   0.4s
[CV] END alpha=0.1, fit_intercept=False, max_iter=1000, solver=cholesky; total time=   0.3s
[CV] END alpha=

### Best regression

In [16]:
dt = Ridge(alpha=10, fit_intercept= True, max_iter= 100, solver= 'lsqr', random_state=21)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
rmse = root_mean_squared_error(y_test,y_pred)
print('RMSE',rmse)

RMSE 1.231697439678581


## Classification

In [61]:
y_clas = y.round().astype(int)
X_train, X_test, y_train, y_test = train_test_split(X,y_clas, test_size=0.2, random_state=21, stratify=y_clas)

### Dummy

In [18]:
dummy_cl = DummyClassifier(strategy='most_frequent')
dummy_cl.fit(X_train,y_train)
dummy_ac = accuracy_score(y_test, dummy_cl.predict(X_test))
dummy_ac

0.6665727170236753

In [19]:
models_class = {'Logistic Regression': LogisticRegression(max_iter=1000),
                'KNN': KNeighborsClassifier(),
                'Decision Tree': DecisionTreeClassifier(random_state=21),
                'SVM': SVC()}
result=[]
for name, model in models_class.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
    score = scores.mean()
    result.append({'Model': name, 'Accuracy': score, 'std': scores.std()})
result_df = pd.DataFrame(result).sort_values(by='Accuracy',ascending=False)
result_df

,Model,Accuracy,std
3,SVM,0.680293,0.000497
0,Logistic Regression,0.676346,0.003425
1,KNN,0.605724,0.003644
2,Decision Tree,0.503313,0.008214


In [20]:
param_grid = {'C': [0.1, 1, 10],
              'gamma': ['scale', 'auto'],
              'kernel': ['linear']}
sv = SVC(random_state=21)
grid_search = GridSearchCV(sv, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
print('Best Parameters', grid_search.best_params_)
print("Best Accuracy", grid_search.best_score_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.8min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.8min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.8min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.8min
[CV] END ...................C=0.1, gamma=auto, kernel=linear; total time= 1.8min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.8min
[CV] END ...................C=0.1, gamma=auto, kernel=linear; total time= 1.8min
[CV] END ...................C=0.1, gamma=auto, kernel=linear; total time= 1.8min
[CV] END ...................C=0.1, gamma=auto, kernel=linear; total time= 1.8min
[CV] END ...................C=0.1, gamma=auto, kernel=linear; total time= 1.8min
[CV] END ....................C=1, gamma=scale, kernel=linear; total time= 2.1min
[CV] END ....................C=1, gamma=scale, ke

### Best classifier by round rate

In [21]:
best_dt = grid_search.best_estimator_
test_ac = best_dt.score(X_test, y_test)
print(f'Test Accuracy', test_ac)

Test Accuracy 0.6744644870349493


### Classify 3 classes

In [62]:
def rate_cat(rate):
    if rate in [0,1]:
        return 'bad'
    elif rate in [2,3]:
        return 'so-so'
    else:
        return 'great'

y_class3 = y_clas.apply(rate_cat)
X_train, X_test, y_train, y_test = train_test_split(X, y_class3, test_size=0.2, random_state=21, stratify=y_class3)


In [23]:
models_class = {'Logistic Regression': LogisticRegression(max_iter=1000),
                'KNN': KNeighborsClassifier(),
                'Decision Tree': DecisionTreeClassifier(random_state=21),
                'SVM': SVC(),}
result=[]
for name, model in models_class.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    score = scores.mean()
    result.append({'Model': name, 'Accuracy': score, 'std': scores.std()})
result_df = pd.DataFrame(result).sort_values(by='Accuracy',ascending=True)
result_df

,Model,Accuracy,std
2,Decision Tree,0.676204,0.008857
1,KNN,0.765013,0.005655
0,Logistic Regression,0.803637,0.002909
3,SVM,0.804624,0.001317


In [24]:
param_grid = {'C': [0.1, 1],
              'gamma': ['scale', 'auto'],
              'kernel': ['rbf', 'linear']}
sv = SVC(random_state=21)
grid_search = GridSearchCV(sv, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
print('Best Parameters', grid_search.best_params_)
print("Best Accuracy", grid_search.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.1min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.1min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.4min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.4min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.4min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.6min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.6min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.0min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.0min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.1min
[CV] END ......................C=0.1, gamma=auto, kernel=rbf; total time= 1.0min
[CV] END ......................C=0.1, gamma=auto,

### Best classifier for 3 classes

In [25]:
best_dt = grid_search.best_estimator_
test_ac = best_dt.score(X_test, y_test)
print(f'Test Accuracy', test_ac)

Test Accuracy 0.8058060879368658


### Bad or Good

In [26]:
result=[]
for name, model in models_class.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1_weighted',n_jobs=-1)
    result.append({'Model': name, 'F1 score': score, 'std': scores.std()})
result_df = pd.DataFrame(result).sort_values(by='F1 score',ascending=False)
result_df

,Model,F1 score,std
0,Logistic Regression,0.804624,0.004118
1,KNN,0.804624,0.004888
2,Decision Tree,0.804624,0.004865
3,SVM,0.804624,0.002959


In [27]:
param_grid = {'C': [0.1, 1, 10],
              'gamma': ['scale', 'auto'],
              'kernel': ['rbf', 'linear']}
sv = SVC(random_state=21)
grid_search = GridSearchCV(sv, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
print('Best Parameters', grid_search.best_params_)
print("Best Accuracy", grid_search.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.0min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.0min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.4min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.4min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.4min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.6min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.7min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time=  58.7s
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.0min
[CV] END ......................C=0.1, gamma=auto, kernel=rbf; total time= 1.0min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.1min
[CV] END ......................C=0.1, gamma=auto

In [28]:
best_sv = grid_search.best_estimator_
predf1 = best_sv.predict(X_test)
print(f'Test Accuracy', f1_score(y_test, predf1, average='weighted', zero_division=0))
print(classification_report(y_test, predf1))

Test Accuracy 0.7326221656796443
              precision    recall  f1-score   support

         bad       0.72      0.14      0.23       342
       great       0.81      0.99      0.89      2826
       so-so       0.00      0.00      0.00       380

    accuracy                           0.81      3548
   macro avg       0.51      0.38      0.38      3548
weighted avg       0.71      0.81      0.73      3548



/home/ivan/projects/DSboot/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ivan/projects/DSboot/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ivan/projects/DSboot/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [29]:
cost_matrix = {('bad', 'great'): 5, 
               ('great', 'bad'): 1,
               ('so-so', 'bad'): 2,
               ('bad', 'so-so'): 3}
def custom_cost(y_true, y_pred):
    total_cost = 0
    for true, pred in zip(y_true, y_pred):
        total_cost += cost_matrix.get((true, pred), 0)
    return -total_cost

cost_scorer = make_scorer(custom_cost, greater_is_better=False)

In [30]:
param_grid = {'C': [0.1, 1, 10],
              'gamma': ['scale', 'auto'],
              'kernel': ['rbf', 'linear']}
sv = SVC(random_state=21)
grid_search = GridSearchCV(sv, param_grid, cv=5, scoring=cost_scorer, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
print('Best Parameters', grid_search.best_params_)
print("Best Accuracy", grid_search.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.1min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.1min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.4min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.5min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.5min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.7min
[CV] END .....................C=0.1, gamma=scale, kernel=rbf; total time= 2.8min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.0min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.0min
[CV] END ......................C=0.1, gamma=auto, kernel=rbf; total time= 1.0min
[CV] END ..................C=0.1, gamma=scale, kernel=linear; total time= 1.1min
[CV] END ......................C=0.1, gamma=auto

## Ensemble

In [31]:
label_map = { 'bad': 0, 'so-so': 1, 'great': 2}
y_enc = y_class3.map(label_map)
X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.2, random_state=21, stratify=y_enc)

In [32]:
models_class = {'Cat Boost': CatBoostClassifier(random_state=21, verbose=0),
                'Random Forest': RandomForestClassifier(random_state=21),
                'XGBoost': XGBClassifier(random_state=21, eval_metric='mlogloss'),
                'Gradient Boosting': GradientBoostingClassifier(random_state=21)}
result=[]
for name, model in models_class.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy',n_jobs=-1)
    score = scores.mean()
    result.append({'Model': name, 'Accuracy': score, 'std': scores.std()})
result_df = pd.DataFrame(result).sort_values(by='Accuracy',ascending=False)
result_df

,Model,Accuracy,std
0,Cat Boost,0.804624,0.003059
2,XGBoost,0.803707,0.001544
3,Gradient Boosting,0.803003,0.001803
1,Random Forest,0.787074,0.001660


In [33]:
param_grid = {'n_estimators': [50, 100, 200],
              'max_depth': [3, 5, 7]}
sv = XGBClassifier(random_state=21)
grid_search = GridSearchCV(sv, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
print('Best Parameters', grid_search.best_params_)
print("Best Accuracy", grid_search.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END .......................max_depth=3, n_estimators=50; total time=   2.2s
[CV] END .......................max_depth=3, n_estimators=50; total time=   2.1s
[CV] END .......................max_depth=3, n_estimators=50; total time=   2.3s
[CV] END .......................max_depth=3, n_estimators=50; total time=   2.2s
[CV] END .......................max_depth=3, n_estimators=50; total time=   2.4s
[CV] END ......................max_depth=3, n_estimators=100; total time=   3.3s
[CV] END ......................max_depth=3, n_estimators=100; total time=   3.5s
[CV] END ......................max_depth=3, n_estimators=100; total time=   3.6s
[CV] END ......................max_depth=3, n_estimators=100; total time=   3.5s
[CV] END ......................max_depth=3, n_estimators=100; total time=   3.8s
[CV] END ......................max_depth=3, n_estimators=200; total time=   5.7s
[CV] END .......................max_depth=5, n_es

### Best ensemble

In [34]:
best_dt = grid_search.best_estimator_
test_ac = best_dt.score(X_test, y_test)
print(f'Test Accuracy', test_ac)

Test Accuracy 0.8055242390078917


## Chose model

In [63]:
svc = SVC(random_state=21, C=1, gamma= 'scale', kernel= 'rbf')
svc.fit(X_train,y_train)
score = svc.score(X_test, y_test)
print(score)

0.8058060879368658


In [64]:
final = svc.fit(X,y_class3)
joblib.dump(final, 'classifcation_model.joblib')

['classifcation_model.joblib']

## PARSER

In [38]:
BASE_URL = 'https://www.epicurious.com/search?'
PARAMETERS = 'q='
PARAMETERS2 = '&page=1&content=recipe'
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

In [39]:
recipes_list = data['title'].tolist()

In [40]:
def normalize_title(title):
    title = title.replace('\"', '').strip().replace(',','%2C').replace(' ', '+')
    return title

In [41]:
def normalize_text(text):
    return ' '.join(text.strip().lower().split())

In [42]:
%time

async def fetch_recipe(title):
    async with httpx.AsyncClient(timeout=5.0) as client:
        try:
            norm_title = normalize_title(title)
            url = f'{BASE_URL}{PARAMETERS}{norm_title}{PARAMETERS2}'
            response = await client.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            all_h2 = soup.find_all('h2')
            found_h2 = None
            search_title = normalize_text(title)
            for h2 in all_h2:
                if normalize_text(h2.get_text()) == search_title:
                    found_h2 = h2
                    break
            if not found_h2:
                for h2 in all_h2:
                    if search_title in normalize_text(h2.get_text()):
                        found_h2 = h2
                        break
            if found_h2:
                parent_a = found_h2.find_parent('a', href=True)
                if parent_a:
                    recipe_url = f"https://www.epicurious.com{parent_a['href']}"
                    return title, recipe_url
                else:
                    return title, None
            else:
                return title, None 
        except Exception as e:
            print(f"Error for '{title}': {str(e)}")
            return title, None

async def process_recipes(recipes):
    result_ok = {}
    result_blank = []
    result_wrong_search = []
    semaphore = asyncio.Semaphore(5)
    async def process(title):
        async with semaphore:
            title, url = await fetch_recipe(title)
            if url:
                result_ok[title] = url
            else:
                result_blank.append(title)
            await asyncio.sleep(0.1) 
    with tqdm(total=len(recipes), desc="Обработка рецептов") as pbar:
        tasks = []
        for title in recipes:
            task = asyncio.create_task(process(title))
            task.add_done_callback(lambda _: pbar.update(1))
            tasks.append(task)
        await asyncio.gather(*tasks)
    return result_ok, result_blank, result_wrong_search
recipes = recipes_list #[:10] для теста
result_ok, result_blank, result_wrong_search = await process_recipes(recipes)

CPU times: user 4 μs, sys: 0 ns, total: 4 μs
Wall time: 8.58 μs


Обработка рецептов:   0%|          | 0/17736 [00:00<?, ?it/s]

Error for 'Beef Tenderloin with Garlic and Brandy ': 
Error for 'Banana Split with Curried Chocolate-Coconut Sauce ': 
Error for 'Sauteed Fennel and Carrots ': 
Error for 'Mocha Brownies ': 
Error for 'Ginger Crunch Cake with Strawberry Sauce ': 
Error for 'Court Bouillon ': 
Error for 'Fennel, Watercress, and Endive Salad with Toasted Pine Nuts ': 
Error for 'Waldorf Salad Celery Boats ': 
Error for 'Roasted-Almond Thumbprints ': 
Error for 'Butternut Squash Latkes with Sage and Pine Nut Yogurt Sauce ': 
Error for 'Honey Pan Dulce with Nuts and Dried Fruit ': 
Error for 'Asparagus, Cucumber, and Sugar Snap Peas with Herb Garlic Dip ': 
Error for 'Lemon Sabayon with Grapefruit ': 
Error for 'Duck Salad with Cheese Toasts and Port-Currant Sauce ': 
Error for 'Grilled Fish Fillets with Lentil Salsa ': 
Error for 'BA's Best Carrot Cake ': 
Error for 'Sour Cream Maple Cake with Lemon Glaze ': 
Error for 'Pasta with Fried Eggs, Caramelized Onions, and Bacon ': 
Error for 'Best-Ever Egg Sala

In [43]:
print("Успешные:", len(result_ok))
print("Пустые ссылки:", len(result_blank))
print("Неправильный поиск:", len(result_wrong_search))

Успешные: 15622
Пустые ссылки: 2114
Неправильный поиск: 0


In [44]:
with open("../datasets/data.json", "w", encoding="utf-8") as file:
    json.dump(result_ok, file, indent=4, ensure_ascii=False)

In [79]:
with open('../datasets/data.json', 'r') as f:
	urls_list = json.load(f)
data["url"] = data["title"].map(urls_list)
data.to_csv('../datasets/recipes.csv')

/tmp/ipykernel_3673/451626086.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["url"] = data["title"].map(urls_list)


In [81]:
data = pd.read_csv('../datasets/recipes.csv')
count_with_url = data['url'].notna().sum()
print(f"Количество записей с URL: {count_with_url}")

Количество записей с URL: 15628


In [85]:
datac = data.dropna(subset=['url'])
print(f"Очищенный DataFrame: {len(datac)} записей")

Очищенный DataFrame: 15628 записей


In [86]:
datac.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15628 entries, 0 to 17735
Columns: 382 entries, Unnamed: 0 to url
dtypes: float64(379), int64(1), object(2)
memory usage: 45.7+ MB


In [90]:
datac.to_csv('../datasets/recipes.csv')

In [91]:
df = pd.read_csv('../datasets/recipes.csv')
title_idx = df.columns.get_loc("title")
df = df.iloc[:, title_idx:]
df.to_csv('../datasets/recipes.csv',index=False)

In [92]:
df

,title,rating,almond,amaretto,anchovy,anise,aperitif,apple,apple juice,apricot,...,whiskey,white wine,whole wheat,wild rice,wine,yellow squash,yogurt,zucchini,turkey,url
0,"Lentil, Apple, and Turkey Wrap",2.500,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,https://www.epicurious.com/recipes/food/views/...
1,Boudin Blanc Terrine with Red Onion Confit,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
2,Potato and Fennel Soup Hodge,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
3,Mahi-Mahi in Tomato Olive Sauce,5.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
4,Spinach Noodle Casserole,3.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15623,Chinese Barbecued Spareribs,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
15624,Artichoke and Parmesan Risotto,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...
15625,Turkey Cream Puff Pie,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,https://www.epicurious.com/recipes/food/views/...
15626,Snapper on Angel Hair with Citrus Cream,4.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,https://www.epicurious.com/recipes/food/views/...


In [93]:
ingredients_list = df.columns.tolist()
ingredients_list.remove('url')
print(ingredients_list)
print(len(ingredients_list))
print(df)

['title', 'rating', 'almond', 'amaretto', 'anchovy', 'anise', 'aperitif', 'apple', 'apple juice', 'apricot', 'artichoke', 'arugula', 'asian pear', 'asparagus', 'aspen', 'avocado', 'bacon', 'banana', 'barley', 'basil', 'bass', 'bean', 'beef', 'beef rib', 'beef shank', 'beef tenderloin', 'beer', 'beet', 'bell pepper', 'berry', 'biscuit', 'blackberry', 'blue cheese', 'blueberry', 'bok choy', 'bourbon', 'braise', 'bran', 'brandy', 'bread', 'breadcrumbs', 'brie', 'brine', 'brisket', 'broccoli', 'broccoli rabe', 'broil', 'brown rice', 'brownie', 'brussel sprout', 'bulgur', 'burrito', 'butter', 'buttermilk', 'butternut squash', 'butterscotch/caramel', 'cabbage', 'campari', 'candy', 'cantaloupe', 'capers', 'caraway', 'cardamom', 'carrot', 'cashew', 'cauliflower', 'caviar', 'celery', 'chambord', 'champagne', 'chard', 'chartreuse', 'cheddar', 'cheese', 'cherry', 'chestnut', 'chicken', 'chickpea', 'chile', 'chile pepper', 'chili', 'chive', 'chocolate', 'cilantro', 'cinnamon', 'clam', 'clove', 'co

In [94]:
nutrients = pd.read_csv('../datasets/nutrients_daily_norm.csv')
nutrients_list = nutrients['nutrient'].tolist()
print(nutrients_list)
print(nutrients)

['Fat', 'Saturated Fat', 'Cholesterol', 'Carbohydrate', 'Sodium', 'Dietary Fiber', 'Protein', 'Added Sugars', 'Vitamin A', 'Vitamin C', 'Calcium', 'Iron', 'Vitamin D', 'Vitamin E', 'Vitamin K', 'Vitamin B1', 'Vitamin B2', 'Vitamin B3', 'Vitamin B6', 'Folate', 'Vitamin B12', 'Biotin', 'Vitamin B5', 'Phosphorus', 'Iodine', 'Magnesium', 'Zinc', 'Selenium', 'Copper', 'Manganese', 'Chromium', 'Molybdenum', 'Chloride', 'Potassium', 'Choline']
         nutrient  measure  number
0             Fat        g    78.0
1   Saturated Fat        g    20.0
2     Cholesterol       mg   300.0
3    Carbohydrate        g   275.0
4          Sodium       mg  2300.0
5   Dietary Fiber        g    28.0
6         Protein        g    50.0
7    Added Sugars        g    50.0
8       Vitamin A  mcg RAE   900.0
9       Vitamin C       mg    90.0
10        Calcium       mg  1300.0
11           Iron       mg    18.0
12      Vitamin D      mcg    20.0
13      Vitamin E       mg    15.0
14      Vitamin K      mcg   120.0

In [95]:
ingredients = pd.DataFrame(ingredients_list, columns=['ingredient'])
for nutrient in nutrients_list:
    ingredients[nutrient] = None
print(f'{ingredients.columns.tolist()} \n')
print(ingredients.head())

['ingredient', 'Fat', 'Saturated Fat', 'Cholesterol', 'Carbohydrate', 'Sodium', 'Dietary Fiber', 'Protein', 'Added Sugars', 'Vitamin A', 'Vitamin C', 'Calcium', 'Iron', 'Vitamin D', 'Vitamin E', 'Vitamin K', 'Vitamin B1', 'Vitamin B2', 'Vitamin B3', 'Vitamin B6', 'Folate', 'Vitamin B12', 'Biotin', 'Vitamin B5', 'Phosphorus', 'Iodine', 'Magnesium', 'Zinc', 'Selenium', 'Copper', 'Manganese', 'Chromium', 'Molybdenum', 'Chloride', 'Potassium', 'Choline'] 

  ingredient   Fat Saturated Fat Cholesterol Carbohydrate Sodium  \
0      title  None          None        None         None   None   
1     rating  None          None        None         None   None   
2     almond  None          None        None         None   None   
3   amaretto  None          None        None         None   None   
4    anchovy  None          None        None         None   None   

  Dietary Fiber Protein Added Sugars Vitamin A  ... Magnesium  Zinc Selenium  \
0          None    None         None      None  ...   

In [97]:
NUTRIENT_MAPPING = {
    'Fat': ['Total lipid (fat)'],
    'Saturated Fat': ['Fatty acids, total saturated'],
    'Cholesterol': ['Cholesterol'],
    'Carbohydrate': ['Carbohydrate, by difference'],
    'Dietary Fiber': ['Fiber, total dietary'],
    'Added Sugars': ['Total Sugars'],
    'Protein': ['Protein'],
    'Vitamin A': ['Vitamin A, RAE'],
    'Vitamin C': ['Vitamin C, total ascorbic acid'],
    'Vitamin D': ['Vitamin D (D2 + D3)'],
    'Vitamin E': ['Vitamin E (alpha-tocopherol)'],
    'Vitamin K': ['Vitamin K (phylloquinone)'],
    'Vitamin B1': ['Thiamin'],
    'Vitamin B2': ['Riboflavin'],
    'Vitamin B3': ['Niacin'],
    'Vitamin B6': ['Vitamin B-6'],
    'Folate': ['Folate, DFE'],
    'Vitamin B12': ['Vitamin B-12'],
    'Vitamin B5': ['Pantothenic acid'],
    'Calcium': ['Calcium, Ca'],
    'Iron': ['Iron, Fe'],
    'Magnesium': ['Magnesium, Mg'],
    'Phosphorus': ['Phosphorus, P'],
    'Potassium': ['Potassium, K'],
    'Sodium': ['Sodium, Na'],
    'Zinc': ['Zinc, Zn'],
    'Copper': ['Copper, Cu'],
    'Manganese': ['Manganese, Mn'],
    'Selenium': ['Selenium, Se'],
    'Iodine': ['Iodine, I'],
    'Chloride': ['Chloride, Cl', 'Chloride'],
    'Choline': ['Choline, total'],
    'Biotin': ['Biotin'],
    'Molybdenum': ['Molybdenum, Mo'],
    'Chromium': ['Chromium, Cr', 'Chromium']
}

In [98]:
def normalize_nutrient_name(api_nutrient_name):
    api_nutrient_name = api_nutrient_name.lower()
    if 'added' in api_nutrient_name:
        return None
    
    for our_name, variants in NUTRIENT_MAPPING.items():
        if any(variant.lower() in api_nutrient_name for variant in variants):
            return our_name
    return None

In [100]:
def parse_ingredient_nutrients(ingredient, api_key):
    try:
        response = httpx.get(
            f'https://api.nal.usda.gov/fdc/v1/foods/search',
            params={
                'query': ingredient,
                'dataType': ['Foundation', 'SR Legacy'],
                'pageSize': 1,
                'api_key': api_key
            }
        )
        
        if response.status_code == 200:
            foods = response.json().get('foods', [])
            if foods:
                api_nutrients = foods[0].get('foodNutrients', [])           
                for nutrient in api_nutrients:
                    norm_name = normalize_nutrient_name(nutrient['nutrientName'])
                    if norm_name:
                        daily_value = nutrients.loc[nutrients['nutrient'] == norm_name]
                        measure = daily_value['measure'].values[0]
                        daily_value = float(daily_value['number'].values[0])
                        percent_daily_value = round((float(nutrient['value']) / daily_value * 100), 2)
                        ingredients.loc[ingredients['ingredient'] == ingredient, norm_name] = percent_daily_value
    except Exception as e:
        print(f"Ошибка при обработке {ingredient}: {(e)}")
    return

In [101]:
%time

API_KEY = "DpAqxoXGgCv9MIlouNyMQQwSHv2Iwuwl2HmOYyGE"
for ingredient in tqdm(ingredients_list):
    result = parse_ingredient_nutrients(ingredient, API_KEY)
print(ingredients.info())
print(ingredients)

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 5.72 μs


  0%|          | 0/380 [00:00<?, ?it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ingredient     380 non-null    object
 1   Fat            319 non-null    object
 2   Saturated Fat  288 non-null    object
 3   Cholesterol    278 non-null    object
 4   Carbohydrate   319 non-null    object
 5   Sodium         315 non-null    object
 6   Dietary Fiber  294 non-null    object
 7   Protein        319 non-null    object
 8   Added Sugars   239 non-null    object
 9   Vitamin A      268 non-null    object
 10  Vitamin C      287 non-null    object
 11  Calcium        317 non-null    object
 12  Iron           317 non-null    object
 13  Vitamin D      223 non-null    object
 14  Vitamin E      227 non-null    object
 15  Vitamin K      213 non-null    object
 16  Vitamin B1     285 non-null    object
 17  Vitamin B2     285 non-null    object
 18  Vitamin B3     286 non-null   

In [103]:
ingredients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ingredient     380 non-null    object
 1   Fat            319 non-null    object
 2   Saturated Fat  288 non-null    object
 3   Cholesterol    278 non-null    object
 4   Carbohydrate   319 non-null    object
 5   Sodium         315 non-null    object
 6   Dietary Fiber  294 non-null    object
 7   Protein        319 non-null    object
 8   Added Sugars   239 non-null    object
 9   Vitamin A      268 non-null    object
 10  Vitamin C      287 non-null    object
 11  Calcium        317 non-null    object
 12  Iron           317 non-null    object
 13  Vitamin D      223 non-null    object
 14  Vitamin E      227 non-null    object
 15  Vitamin K      213 non-null    object
 16  Vitamin B1     285 non-null    object
 17  Vitamin B2     285 non-null    object
 18  Vitamin B3     286 non-null   

In [104]:
ingredients.columns[1:]

Index(['Fat', 'Saturated Fat', 'Cholesterol', 'Carbohydrate', 'Sodium',
       'Dietary Fiber', 'Protein', 'Added Sugars', 'Vitamin A', 'Vitamin C',
       'Calcium', 'Iron', 'Vitamin D', 'Vitamin E', 'Vitamin K', 'Vitamin B1',
       'Vitamin B2', 'Vitamin B3', 'Vitamin B6', 'Folate', 'Vitamin B12',
       'Biotin', 'Vitamin B5', 'Phosphorus', 'Iodine', 'Magnesium', 'Zinc',
       'Selenium', 'Copper', 'Manganese', 'Chromium', 'Molybdenum', 'Chloride',
       'Potassium', 'Choline'],
      dtype='object')

In [105]:
nutrition_cols = ingredients.columns[1:]
ingredients[nutrition_cols] = ingredients[nutrition_cols].apply(pd.to_numeric, errors='coerce')

In [106]:
ingredients = ingredients.fillna(0)
ingredients

,ingredient,Fat,Saturated Fat,Cholesterol,Carbohydrate,Sodium,Dietary Fiber,Protein,Added Sugars,Vitamin A,...,Magnesium,Zinc,Selenium,Copper,Manganese,Chromium,Molybdenum,Chloride,Potassium,Choline
0,title,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00
1,rating,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00
2,almond,64.36,0.00,0.0,5.89,0.04,33.11,52.40,0.00,0.00,...,59.76,25.45,1.35,108.33,78.70,0.0,80.22,0.0,14.19,0.00
3,amaretto,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00
4,anchovy,6.21,6.40,20.0,0.00,4.52,0.00,40.80,0.00,1.67,...,9.76,15.64,66.36,23.44,3.04,0.0,0.00,0.0,8.15,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,wine,0.00,0.00,0.0,0.10,0.35,0.00,0.08,0.00,0.00,...,0.95,0.27,0.00,1.11,2.00,0.0,0.00,0.0,0.83,0.00
376,yellow squash,0.17,0.00,0.0,1.60,0.00,3.41,1.78,0.00,0.00,...,4.00,1.85,0.00,6.76,10.43,0.0,0.00,0.0,4.68,0.00
377,yogurt,2.31,1.29,0.0,5.82,1.52,0.71,7.00,2.48,0.22,...,9.52,2.82,23.64,8.33,0.00,0.0,0.00,0.0,1.00,8.80
378,zucchini,0.51,0.41,0.0,1.13,0.13,3.93,5.42,0.00,2.78,...,7.86,7.55,0.55,10.78,8.52,0.0,0.00,0.0,9.77,0.00


In [107]:
ingredients.to_csv('../datasets/ingredients_nutrients.csv', index=False)

In [108]:
tst = pd.read_csv('../datasets/ingredients_nutrients.csv')
tst

,ingredient,Fat,Saturated Fat,Cholesterol,Carbohydrate,Sodium,Dietary Fiber,Protein,Added Sugars,Vitamin A,...,Magnesium,Zinc,Selenium,Copper,Manganese,Chromium,Molybdenum,Chloride,Potassium,Choline
0,title,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00
1,rating,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00
2,almond,64.36,0.00,0.0,5.89,0.04,33.11,52.40,0.00,0.00,...,59.76,25.45,1.35,108.33,78.70,0.0,80.22,0.0,14.19,0.00
3,amaretto,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00
4,anchovy,6.21,6.40,20.0,0.00,4.52,0.00,40.80,0.00,1.67,...,9.76,15.64,66.36,23.44,3.04,0.0,0.00,0.0,8.15,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,wine,0.00,0.00,0.0,0.10,0.35,0.00,0.08,0.00,0.00,...,0.95,0.27,0.00,1.11,2.00,0.0,0.00,0.0,0.83,0.00
376,yellow squash,0.17,0.00,0.0,1.60,0.00,3.41,1.78,0.00,0.00,...,4.00,1.85,0.00,6.76,10.43,0.0,0.00,0.0,4.68,0.00
377,yogurt,2.31,1.29,0.0,5.82,1.52,0.71,7.00,2.48,0.22,...,9.52,2.82,23.64,8.33,0.00,0.0,0.00,0.0,1.00,8.80
378,zucchini,0.51,0.41,0.0,1.13,0.13,3.93,5.42,0.00,2.78,...,7.86,7.55,0.55,10.78,8.52,0.0,0.00,0.0,9.77,0.00
